In [26]:
!pip install transformers torch datasets
from transformers import BertTokenizer, BertModel, pipeline, AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments, AutoModelForQuestionAnswering
from datasets import load_dataset, Dataset
import torch
import torch.nn.functional as F
import re
from nltk.tokenize import sent_tokenize
import nltk
import json
import os
import numpy as np
os.environ["WANDB_DISABLED"] = "true"
nltk.download('punkt')
nltk.download('punkt_tab')
from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
def load_text(file_path):
    """
    Load text from a file.
    """
    print("Loading the text from the file...")
    with open(book_path, 'r', encoding='utf-8') as file:
        return file.read()

def remove_gutenberg_header_footer(text, start_marker, end_marker):
    """
    Remove the header and footer from Project Gutenberg text.

    Finds specified start and end markers and returns the content in between.

    Parameters:
        text (str): The text to clean.
        start_marker (str): Marker indicating the start of main content.
        end_marker (str): Marker indicating the end of main content.

    Returns:
        str: Text with header and footer removed, or original text if markers are not found.
    """
    start_index = text.find(start_marker)
    if start_index == -1:
        return text

    end_index = text.find(end_marker)
    if end_index == -1:
        return text

    cleaned_text = text[start_index + len(start_marker):end_index].strip()
    return cleaned_text

book_path = "/content/drive/My Drive/Cosc524 - Collaboration/a study in scarlet.txt"

# here we are cleaning the text and removing the headers and footers
raw_text = load_text(book_path)

start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"

cleaned_text = remove_gutenberg_header_footer(raw_text, start_marker, end_marker)

print("Cleaned text sample:\n")
print(cleaned_text[:500])

def tokenize_sentences(text):
    """
    Tokenize the input text into sentences using NLTK.

    Parameters:
        text (str): The text to tokenize.

    Returns:
        list: A list of tokenized sentences.
    """
    print("Tokenizing the text into sentences...")
    sentences = sent_tokenize(text)
    print(f"Total sentences: {len(sentences)}")
    return sentences

my_sentences = tokenize_sentences(cleaned_text)

Loading the text from the file...
Cleaned text sample:

A STUDY IN SCARLET

By A. Conan Doyle




CONTENTS

 A STUDY IN SCARLET.

 PART I.
 CHAPTER I. MR. SHERLOCK HOLMES.
 CHAPTER II. THE SCIENCE OF DEDUCTION.
 CHAPTER III. THE LAURISTON GARDENS MYSTERY
 CHAPTER IV. WHAT JOHN RANCE HAD TO TELL.
 CHAPTER V. OUR ADVERTISEMENT BRINGS A VISITOR.
 CHAPTER VI. TOBIAS GREGSON SHOWS WHAT HE CAN DO.
 CHAPTER VII. LIGHT IN THE DARKNESS.

 PART II. THE COUNTRY OF THE SAINTS
 CHAPTER I. ON THE GREAT ALKALI PLAIN.
 CHAPTER II. THE FLOWER OF UTAH.
 CHAPTER III. J
Tokenizing the text into sentences...
Total sentences: 2208


In [10]:
# # For text generation
# generator = pipeline('fill-mask', model='bert-base-uncased')

# # For text retrieval

# #I should try swapping to the question answer BERT model!
# retriever_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# retriever_model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [81]:
# def retrieve_relevant_info(query, top_k=5, embeddings=None, sentences=None):

#   """Retrieves the most relevant sentences based on cosine similarity.

#   Args:
#     query: The input query string.
#     top_k: The number of top sentences to retrieve.

#   Returns:
#     A list of the top_k most relevant sentences.
#   """
#   query_embedding = retriever_model(**retriever_tokenizer(query, return_tensors="pt")).last_hidden_state[:, 0, :]
#   similarities = F.cosine_similarity(embeddings, query_embedding, dim=1) # Assume 'embeddings' is pre-calculated
#   top_indices = torch.topk(similarities, top_k).indices
#   relevant_sentences = [sentences[idx] for idx in top_indices]  # Assume 'sentences' is pre-calculated
#   return relevant_sentences

In [82]:
# def generate_text_with_rag(prompt, top_k=5, embeddings=None, sentences=None):
#   """Generates text using RAG with BERT.

#   Args:
#     prompt: The input prompt for text generation.
#     top_k: The number of retrieved sentences to use for augmentation.

#   Returns:
#     The generated text.
#   """
#   relevant_info = retrieve_relevant_info(prompt, top_k=top_k, embeddings=embeddings, sentences=sentences)
#   augmented_prompt = f"{prompt} [SEP] {' '.join(relevant_info)} [MASK]."  # Add [MASK] token
#   generated_text = generator(augmented_prompt)[0]['sequence']
#   return generated_text

Load word embeddings

In [28]:
my_embeddings = torch.load('/content/drive/My Drive/Cosc524 - Collaboration/embeddings/bert_embeddings.pt')
# prompt = "Who was the murderer?"
# generated_text = generate_text_with_rag(prompt=prompt, top_k=5, embeddings=my_embeddings, sentences=my_sentences)
# # print(generated_text)
# for sentence in sent_tokenize(generated_text):
#   print(sentence)

<ipython-input-28-e99a740c6d6a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  my_embeddings = torch.load('/content/drive/My Drive/Cosc524 - Collaboration/embeddings/bert_

Working on RAG doot doot, doot doot

In [29]:
generator = pipeline('question-answering', model='distilbert-base-uncased-distilled-squad')

def generate_text_with_rag(prompt, top_k=5, embeddings=None, sentences=None):
    relevant_info = retrieve_relevant_info(prompt, top_k=top_k, embeddings=embeddings, sentences=sentences)
    context = " ".join(relevant_info)  # Combine retrieved sentences as context
    generated_text = generator(question=prompt, context=context)['answer']
    return generated_text

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [30]:
# Retrieve relevant sentences based on query
def retrieve_relevant_info(query, top_k=5, embeddings=None, sentences=None):
    """
    Retrieves the most relevant sentences based on cosine similarity with the query.

    Args:
        query: Input query string.
        top_k: Number of top sentences to retrieve.
        embeddings: Precomputed sentence embeddings.
        sentences: List of sentences corresponding to the embeddings.

    Returns:
        A list of the top_k most relevant sentences.
    """
    # Compute query embedding
    query_input = retriever_tokenizer(query, return_tensors="pt", truncation=True)
    with torch.no_grad():
        query_output = retriever_model(**query_input)
        query_embedding = query_output.last_hidden_state.mean(dim=1)  # Average over tokens

    # Compute cosine similarities
    similarities = F.cosine_similarity(embeddings, query_embedding, dim=1)

    # Get top-k most similar sentences
    top_indices = torch.topk(similarities, top_k).indices
    relevant_sentences = [sentences[idx] for idx in top_indices]
    return relevant_sentences


Generate RAG context

In [31]:
# For text generation
generator = pipeline('fill-mask', model='bert-base-uncased')

#I should try swapping to the question answer BERT model!
retriever_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
retriever_model = BertModel.from_pretrained('bert-base-uncased')

# Example usage
query = "Who is the murderer in A Study in Scarlet?"
relevant_sentences = retrieve_relevant_info(query, top_k=5, embeddings=my_embeddings, sentences=my_sentences)
context = ' '.join(relevant_sentences)
# print("Relevant Sentences:", relevant_sentences)
for sentence in relevant_sentences:
  print(sentence)

#The context looks ok here.
print(len(context))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Are they blood stains, or mud stains, or rust stains, or fruit
stains, or what are they?
A man is
suspected of a crime months perhaps after it has been committed.
What has become of
the cabman who drove them?
The
tradesman was put to his trade and the artisan to his calling.
JOHN FERRIER TALKS WITH THE PROPHET.
312


In [20]:
#truncate context to match BERT context window size
#The model I am using is 512 window size
def truncate_context(question, context, tokenizer, max_length=512):
    """
    Truncates the context to fit within the model's max token limit.

    Args:
        question: The input question.
        context: The input context string.
        tokenizer: The tokenizer used to calculate token length.
        max_length: The maximum allowed length of tokens.

    Returns:
        A truncated context string.
    """
    tokens = tokenizer(question, context, truncation=True, max_length=max_length, return_tensors="pt")
    return tokenizer.decode(tokens["input_ids"][0], skip_special_tokens=True)

print(len(context))

312


In [32]:
fine_tuned_model_path = "/content/drive/My Drive/Cosc524 - Collaboration/qa_model/finetuned"
dataset_path = "/content/drive/My Drive/Cosc524 - Collaboration/dataset/qa_dataset.json"

print("Loading the fine-tuned model...")
fine_tuned_model = AutoModelForQuestionAnswering.from_pretrained(fine_tuned_model_path)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Initialize tokenizer and QA pipeline
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
qa_pipeline = pipeline("question-answering", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

query = "Who is the murderer in A Study in Scarlet? Was it a cat?"

#If we change the tokenizer in the parameters it generates an incorrect text answer... but atleast it isnt just a comma. ;-;
context = truncate_context(query, context, fine_tuned_tokenizer, max_length=512)

# Get the answer
answer = qa_pipeline(question=query, context=context)["answer"]
print('\n\n')
print(answer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Loading the fine-tuned model...



tradesman was put to
